# ModelMemberGraph and Serialization

Example notebook of ModelMemberGraph functionality

In [1]:
import numpy as np

import pygsti

In [2]:
from pygsti.modelpacks import smq2Q_XYICNOT

# Similar/Equivalent

In [3]:
ex_mdl1 = smq2Q_XYICNOT.target_model()
ex_mdl2 = ex_mdl1.copy()

In [4]:
ex_mmg1 = ex_mdl1.create_modelmember_graph()

In [5]:
ex_mmg1.print_graph()

Modelmember type: preps
  rho0: FullState (0)
Modelmember type: povms
  Mdefault: UnconstrainedPOVM (1)
Modelmember type: operations
  []: FullArbitraryOp (2)
  Gxpi2:1: FullArbitraryOp (3)
  Gypi2:1: FullArbitraryOp (4)
  Gxpi2:0: FullArbitraryOp (5)
  Gypi2:0: FullArbitraryOp (6)
  Gcnot:0:1: FullArbitraryOp (7)
Modelmember type: instruments
Modelmember type: factories


In [6]:
ex_mmg1.mm_nodes['operations']['Gxpi2', 0]

In [7]:
ex_mmg2 = ex_mdl2.create_modelmember_graph()
print(ex_mmg1.is_similar(ex_mmg2))
print(ex_mmg1.is_equivalent(ex_mmg2))

True
True


In [8]:
ex_mdl2.operations['Gxpi2', 0][0, 0] = 0.0
ex_mmg2 = ex_mdl2.create_modelmember_graph()
print(ex_mmg1.is_similar(ex_mmg2))
print(ex_mmg1.is_equivalent(ex_mmg2))

True
False


In [9]:
ex_mdl2.operations['Gxpi2', 0] = pygsti.modelmembers.operations.StaticArbitraryOp(ex_mdl2.operations['Gxpi2', 0])
ex_mmg2 = ex_mdl2.create_modelmember_graph()
print(ex_mmg1.is_similar(ex_mmg2))
print(ex_mmg1.is_equivalent(ex_mmg2))

False
False


In [10]:
pspec = pygsti.processors.QubitProcessorSpec(2, ['Gi', 'Gxpi2', 'Gypi2', 'mygate'], geometry='line', nonstd_gate_unitaries={'mygate': np.eye(2, dtype='complex')})

ln_mdl1 = pygsti.models.create_crosstalk_free_model(pspec,
                                                    depolarization_strengths={('Gxpi2', 0): 0.1, ('mygate', 0): 0.2},
                                                    lindblad_error_coeffs={('Gypi2', 1): {('H', 1): 0.2, ('S', 2): 0.3}})
print(ln_mdl1)

layers:rho0 = Computational Z-basis state vec for 2 qubits w/z-values: [0 0]

layers:Mdefault = Computational(Z)-basis POVM on 2 qubits and filter None


layers:(auto_global_idle) = 
Composed operation of 2 factors:
Factor 0:
Embedded operation with full dimension 16 and state space QubitSpace((0, 1))
 that embeds the following 4-dimensional operation into acting on the (0,) space
StaticStandardOp with name Gi and evotype densitymx
Factor 1:
Embedded operation with full dimension 16 and state space QubitSpace((0, 1))
 that embeds the following 4-dimensional operation into acting on the (1,) space
StaticStandardOp with name Gi and evotype densitymx


layers:Gi:0 = 
Embedded operation with full dimension 16 and state space QubitSpace((0, 1))
 that embeds the following 4-dimensional operation into acting on the (0,) space
StaticStandardOp with name Gi and evotype densitymx


layers:Gi:1 = 
Embedded operation with full dimension 16 and state space QubitSpace((0, 1))
 that embeds the follow

In [11]:
ln_mmg1 = ln_mdl1.create_modelmember_graph()
ln_mmg1.print_graph()

Modelmember type: prep_blks_layers
  rho0: ComputationalBasisState (0)
Modelmember type: povm_blks_layers
  Mdefault: ComputationalBasisPOVM (1)
Modelmember type: operation_blks_layers
  (auto_global_idle): ComposedOp (5)
    EmbeddedOp (3)
      StaticStandardOp (2)
    EmbeddedOp (4)
      StaticStandardOp (2)
  Gi:0: EmbeddedOp (6)
    StaticStandardOp (2)
  Gi:1: EmbeddedOp (7)
    StaticStandardOp (2)
  Gxpi2:0: EmbeddedOp (11)
    ComposedOp (10)
      StaticStandardOp (8)
      DepolarizeOp (9)
  Gxpi2:1: EmbeddedOp (12)
    StaticStandardOp (8)
  Gypi2:0: EmbeddedOp (14)
    StaticStandardOp (13)
  Gypi2:1: EmbeddedOp (18)
    ComposedOp (17)
      StaticStandardOp (13)
      ExpErrorgenOp (16)
        LindbladErrorgen (15)
  mygate:0: EmbeddedOp (22)
    ComposedOp (21)
      StaticUnitaryOp (19)
      DepolarizeOp (20)
  mygate:1: EmbeddedOp (23)
    StaticUnitaryOp (19)
Modelmember type: operation_blks_gates
  Gi: StaticStandardOp (2)
  Gxpi2: StaticStandardOp (8)
  Gxpi2:0:

In [12]:
# Should be exactly the same
ln_mdl2 = pygsti.models.create_crosstalk_free_model(pspec,
                                                    depolarization_strengths={('Gxpi2', 0): 0.1},
                                                    lindblad_error_coeffs={('Gypi2', 1): {('H', 1): 0.2, ('S', 2): 0.3}})
ln_mmg2 = ln_mdl2.create_modelmember_graph()
print(ln_mmg1.is_similar(ln_mmg2))
print(ln_mmg1.is_equivalent(ln_mmg2))

False
False


In [13]:
# Should be similar if we change params
ln_mdl3 = pygsti.models.create_crosstalk_free_model(pspec,
                                                    depolarization_strengths={('Gxpi2', 0): 0.01},
                                                    lindblad_error_coeffs={('Gypi2', 1): {('H', 1): 0.5, ('S', 2): 0.1}})
ln_mmg3 = ln_mdl3.create_modelmember_graph()
print(ln_mmg1.is_similar(ln_mmg3))
print(ln_mmg1.is_equivalent(ln_mmg3))

False
False


In [14]:
# Should fail both, depolarize is on different gate
ln_mdl4 = pygsti.models.create_crosstalk_free_model(pspec,
                                                    depolarization_strengths={('Gypi2', 0): 0.1},
                                                    lindblad_error_coeffs={('Gypi2', 1): {('H', 1): 0.2, ('S', 2): 0.3}})
ln_mmg4 = ln_mdl4.create_modelmember_graph()
print(ln_mmg1.is_similar(ln_mmg4))
print(ln_mmg1.is_equivalent(ln_mmg4))

False
False


# Serialization

In [15]:
pygsti.io.write_model_json(ex_mdl1, 'example_files/ex_mdl1.json')

In [16]:
pygsti.io.write_model_json(ln_mdl1, 'example_files/ln_mdl1.json')